In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,14215
2,Huelva,Confirmados PDIA 14 días,2375
3,Huelva,Tasa PDIA 14 días,"465,0088204831001"
4,Huelva,Confirmados PDIA 7 días,1785
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,72
629,Municipio de Huelva sin especificar,Total Confirmados,232
630,Municipio de Huelva sin especificar,Curados,114


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  14215.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4035.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 300 personas en los últimos 7 días 

Un positivo PCR cada 216 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,14215.0,1785.0,2375.0,510743.0,349.490840,465.008820,496.0
Huelva-Costa,8304.0,1140.0,1548.0,288115.0,395.675338,537.285459,371.0
Huelva (capital),4035.0,479.0,666.0,143663.0,333.419182,463.584917,153.0
Isla Cristina,901.0,267.0,356.0,21264.0,1255.643341,1674.191121,81.0
Condado-Campiña,4076.0,400.0,505.0,155057.0,257.969650,325.686683,77.0
Lepe,1144.0,114.0,127.0,27431.0,415.588203,462.979840,41.0
Sierra de Huelva-Andévalo Central,1604.0,173.0,239.0,67571.0,256.026994,353.702032,33.0
Ayamonte,456.0,78.0,96.0,20946.0,372.386136,458.321398,28.0
Aljaraque,580.0,75.0,121.0,21260.0,352.775165,569.143932,24.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,65.0,48.0,55.0,1305.0,3678.160920,4214.559387,6.0
Villalba del Alcor,213.0,95.0,108.0,3338.0,2846.015578,3235.470342,7.0
Villanueva de las Cruces,7.0,7.0,7.0,373.0,1876.675603,1876.675603,0.0
Castaño del Robledo,6.0,3.0,4.0,223.0,1345.291480,1793.721973,0.0
Isla Cristina,901.0,267.0,356.0,21264.0,1255.643341,1674.191121,81.0
Calañas,73.0,22.0,26.0,2791.0,788.247940,931.565747,5.0
Palma del Condado (La),667.0,80.0,91.0,10761.0,743.425332,845.646315,18.0
Alosno,78.0,22.0,29.0,3923.0,560.795310,739.230181,7.0
Cortegana,161.0,25.0,33.0,4666.0,535.790827,707.243892,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,75.0,6.0,20.0,3073.0,195.248942,650.829808,0.0,0.300000
Corteconcepción,21.0,1.0,3.0,548.0,182.481752,547.445255,0.0,0.333333
Zarza-Perrunal (La),17.0,2.0,4.0,1267.0,157.853197,315.706393,1.0,0.500000
San Bartolomé de la Torre,61.0,8.0,16.0,3736.0,214.132762,428.265525,3.0,0.500000
Higuera de la Sierra,17.0,1.0,2.0,1287.0,77.700078,155.400155,0.0,0.500000
Campillo (El),57.0,1.0,2.0,2023.0,49.431537,98.863075,0.0,0.500000
Puebla de Guzmán,44.0,10.0,19.0,3073.0,325.414904,618.288318,3.0,0.526316
Aracena,342.0,21.0,35.0,8107.0,259.035402,431.725669,9.0,0.600000
Aljaraque,580.0,75.0,121.0,21260.0,352.775165,569.143932,24.0,0.619835
